Run the following command:

pip install -r requirements.txt

In [4]:
import re
import tweepy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential,load_model
#from tensorflow.python.keras.utils import _____
from keras.layers import Embedding,LSTM,Dense,Dropout
from sklearn.model_selection import train_test_split

## Importing all the relevant libraries

In [17]:
from dotenv import load_dotenv, dotenv_values
secrets = dotenv_values(".env")

#Imports the credentials for Twitter

consumerKey = secrets["consumerKey"]
consumerSecret = secrets["consumerSecret"]
accessToken = secrets["accessToken"]
accessSecret = secrets["accessSecret"]

In [18]:
authenticate = tweepy.OAuthHandler(consumerKey,consumerSecret)
authenticate.set_access_token(accessToken,accessSecret)

api = tweepy.API(authenticate,wait_on_rate_limit= True)

In [22]:
mention = api.mentions_timeline()
print(mention)

for tweets in mention:
  print(str(tweets.id)+'-'+tweets.text)

[]


In [23]:
dftweet = pd.DataFrame(['Circle K is dead to me. I hate your new slushie machine. #irate #livid'],columns = ["Column2"])
dftweet.head()

,Column2
0,Circle K is dead to me. I hate your new slushi...


In [24]:
#clean tweets
def cleanTxt(text):
  text =re.sub(r'@[A-Za-z0-9]+','',text) #remove @
  text = re.sub(r'#','',text)# remove #
  text = re.sub(r'RT[\s]+','',text)# remove RT
  text = re.sub(r'https?:\/\/\S+','',text)#remove hyperlink

  return text

dftweet['Column2']= dftweet['Column2'].apply(cleanTxt)
print(dftweet)

                                             Column2
0  Circle K is dead to me. I hate your new slushi...


In [27]:
df = pd.read_csv('combined.csv')
df['Column2']= df['Column2'].apply(cleanTxt)
df.head()
df.shape

(3613, 4)

In [28]:
 
tokenizer = Tokenizer(num_words=5000,split =" ")
tokenizer.fit_on_texts(df['Column2'].values)

df_x = tokenizer.texts_to_sequences(df['Column2'].values)
df_x = pad_sequences(df_x)
print(df_x[:5])

tokenizer.fit_on_texts(dftweet['Column2'].values)
x_tweet = tokenizer.texts_to_sequences(dftweet['Column2'].values)
x_tweet = pad_sequences(x_tweet)

print(df_x.shape)

[[   0    0    0    0    0    0    0    0    3   45  335  429 4506 4507
  1803 4508   23 4509 1804  924 4510 4511 4512    5  379 2484  162  831
    59    6   10   98]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0  134    4 4513   13
   430    9 1038  649]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    3  102   26    3  111 1805
   169  154 2485 1435]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    3   43   18  182  114  650   36  323
     2    3  231  564]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    3
    90   10 2486  187]]
(3613, 32)


In [29]:
df_y = pd.get_dummies(df['Column3']).values
[print(df['Column3'][i],df_y[i]) for i in range(0,3)]
print(df_x.shape)
print(df_y.shape)
x_train,x_test,y_train,y_test = train_test_split(df_x,df_y,test_size = 0.2,random_state = 0)

sadness [0 0 0 1]
joy [0 0 1 0]
fear [0 1 0 0]
(3613, 32)
(3613, 4)


In [31]:

model = Sequential()
model.add(Embedding(5000,256,input_length=df_x.shape[1]))
#model.add(Dense(4, activation='softmax'))
model.add(Dropout(0.3))
model.add(LSTM(256,return_sequences=True,dropout=0.3,recurrent_dropout=0.2))
model.add(LSTM(256,dropout=0.3,recurrent_dropout=0.2))
model.add(Dense(4,activation='softmax'))

In [32]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 32, 256)           1280000   
_________________________________________________________________
dropout (Dropout)            (None, 32, 256)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 32, 256)           525312    
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense (Dense)                (None, 4)                 1028      
Total params: 2,331,652
Trainable params: 2,331,652
Non-trainable params: 0
_________________________________________________________________


In [34]:
model.fit(x_train, y_train,epochs=10,batch_size = 32,verbose=2)

Epoch 1/10
91/91 - 35s - loss: 0.1715 - accuracy: 0.9484
Epoch 2/10
91/91 - 33s - loss: 0.1002 - accuracy: 0.9699
Epoch 3/10
91/91 - 34s - loss: 0.0705 - accuracy: 0.9796
Epoch 4/10
91/91 - 32s - loss: 0.0613 - accuracy: 0.9806
Epoch 5/10
91/91 - 32s - loss: 0.0477 - accuracy: 0.9830
Epoch 6/10
91/91 - 31s - loss: 0.0498 - accuracy: 0.9844
Epoch 7/10
91/91 - 29s - loss: 0.0429 - accuracy: 0.9827
Epoch 8/10
91/91 - 34s - loss: 0.0414 - accuracy: 0.9827
Epoch 9/10
91/91 - 34s - loss: 0.0345 - accuracy: 0.9851
Epoch 10/10
91/91 - 29s - loss: 0.0392 - accuracy: 0.9851


In [35]:
model.save('sentiment')

INFO:tensorflow:Assets written to: sentiment\assets


In [84]:
# LOOKS AT THE CONTENT OF THE MENTIONED TWEET
#tweets = api.mentions_timeline(tweet_mode='extended')
#a = tweets[3].in_reply_to_status_id_str
#tweet_a = api.get_status(id=a)
#print(tweet_a.text)
def check (i):
    print(i)
    print(np.max(i))
    a = np.where(i == np.max(i))
    a = a[0][0]
    if ( a == 0):
        return('Anger')
    elif (a == 1):
        return('Fear')
    elif (a ==2):
        return('Joy')
    else:
        return('Sadness')

predictions = model.predict(x_tweet)
[print(df['Column2'][i],predictions[i],y_test[i]) for i in range (0,1)]
print(check(y_test[0]))
print(check(predictions[0]))

I can literally eat creamy pesto pasta topped with grilled chicken, sun dried tomatoes, asparagus and pine nuts every single day of my life [2.7524629e-03 3.2543772e-04 9.9663228e-01 2.8989892e-04] [0 1 0 0]
[0 1 0 0]
1
Fear
[2.7524629e-03 3.2543772e-04 9.9663228e-01 2.8989892e-04]
0.9966323
Joy
